# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symred.utils import unit_n_sphere_cartesian_coords
from symred.symplectic_form import PauliwordOp, QuantumState
from functools import cached_property
from typing import Union

def random_state(num_qubits, num_terms):
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(M,N))
    # note the coefficient vector must be normalized
    coeff_vec = (
        unit_n_sphere_cartesian_coords(np.pi*np.random.rand(M-1)) + 0
        #unit_n_sphere_cartesian_coords(np.pi*np.random.rand(M-1)) * 1j
    )
    return QuantumState(random_state, coeff_vec)

In [6]:
psi = QuantumState(np.array([[1]]))
Y = PauliwordOp(['Y'], [1])
print(psi)
print(Y*psi)

 1.0000000000 |1>
 0.0000000000-1.0000000000j |0>


In [23]:
N = 4 # number of qubits
M = 1 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)
psi_3 = random_state(N, M)

print(psi_1)
print()
print(psi_2)
print()
print(psi_3)

 1.0000000000 |0010>

 1.0000000000 |0110>

 1.0000000000 |0000>


In [24]:
print(psi_1 + psi_2)

 0.7071067812+0.0000000000j |0010> +
 0.7071067812+0.0000000000j |0110>


In [27]:
op  = PauliwordOp(['IXYZ', 'XXXX', 'IYII'],[1, 1,1])
psi = psi_1 + psi_2 + psi_3
print(psi)
print()
new_state = op*psi
print(new_state)

 0.7071067812+0.0000000000j |0000> +
 0.5000000000+0.0000000000j |0010> +
 0.5000000000+0.0000000000j |0110>

 0.0000000000-0.2886751346j |0000> +
 0.0000000000-0.2886751346j |0010> +
 0.0000000000+0.4082482905j |0100> +
-0.0000000000-0.2886751346j |0100> +
 0.0000000000+0.2886751346j |0110> +
 0.0000000000+0.4082482905j |0110> +
 0.2886751346-0.0000000000j |1001> +
 0.2886751346-0.0000000000j |1101> +
 0.4082482905+0.0000000000j |1111>


In [33]:
psi.conjugate * op

TypeError: unsupported operand type(s) for *: 'QuantumState' and 'PauliwordOp'